### Imports

In [ ]:
# Imports
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator


### Data Loading and Pre Processing

In [ ]:
INPUT_SIZE = (224, 224)
BATCH_SIZE = 80
INPUT_SIZE = (224, 224)

# Define FER2013 dataset paths
train_data_dir = 'fer2013/train'
val_data_dir = 'fer2013/val'

# Data generators
train_datagen = ImageDataGenerator(preprocessing_function=None, horizontal_flip=True)
test_datagen = ImageDataGenerator(preprocessing_function=None)

# Load data (change color_mode to 'rgb' since you're converting to RGB in preprocessing)
train_generator = train_datagen.flow_from_directory(
    train_data_dir, target_size=INPUT_SIZE,
    batch_size=BATCH_SIZE, class_mode='categorical', color_mode='rgb'
)

val_generator = test_datagen.flow_from_directory(
    val_data_dir, target_size=INPUT_SIZE,
    batch_size=BATCH_SIZE, class_mode='categorical', color_mode='rgb'
)

# Get one batch of images from the generator
x_batch, y_batch = next(train_generator)

# Check the shape of the first image in the batch
print(x_batch.shape)


### Preparing the Data

In [ ]:
base_model = load_model('mobilenet_face.h5') # face detecton model
print(base_model.input_shape)
net_description = 'mobilenet_face'

def preprocess_fer2013(image):
    # Convert grayscale image to RGB by repeating the single channel across 3 channels
    image = np.repeat(image[..., np.newaxis], 3, axis=-1)  # Correctly expand to (224, 224, 3)
    return image

preprocessing_function = preprocess_fer2013


(None, 224, 224, 3)
Found 28474 images belonging to 7 classes.
Found 7022 images belonging to 7 classes.
(80, 224, 224, 3)


### Train

In [13]:

N_CLASS=val_generator.num_classes
nb_train_samples=train_generator.samples
nb_validation_samples=val_generator.samples
print(N_CLASS,nb_train_samples,nb_validation_samples)

class_to_idx=val_generator.class_indices
idx_to_class={class_to_idx[cls]:cls for cls in class_to_idx}
print(idx_to_class)

(unique, counts) = np.unique(train_generator.classes, return_counts=True)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, idx_to_class, val_generator.class_indices)

layer_name='feats'
#layer_name='global_average_pooling2d_1'
#layer_name='fc7/relu'
layer_out=base_model.get_layer(layer_name) #'global_pooling') #
x=layer_out.output

emotion_preds = Dense(N_CLASS, activation='softmax', name='emotion_preds')(x)
model=Model(base_model.input,emotion_preds)
start_epoch=0

base_model.trainable=False
for l in base_model.layers:
    l.trainable=False
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

print(net_description)

mc = ModelCheckpoint(net_description+'.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
es=EarlyStopping(monitor='val_accuracy',patience=2)
FIRST_EPOCHS=3

print(model.input_shape)

hist1=model.fit(train_generator, steps_per_epoch=nb_train_samples//BATCH_SIZE, epochs=FIRST_EPOCHS, verbose=1, 
                    initial_epoch=0, callbacks=[mc, es], validation_data=val_generator, validation_steps=nb_validation_samples // BATCH_SIZE,class_weight=class_weights)



7 28474 7022
{0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
[3995  436 4097 7214 4965 4596 3171] {0: 1.8057571964956194, 1: 16.545871559633028, 2: 1.7608005857944837, 3: 1.0, 4: 1.4529707955689828, 5: 1.5696257615317666, 6: 2.2749921160517186} {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'} {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
mobilenet_face
(None, 224, 224, 3)
Epoch 1/3
355/355 [==============================] - ETA: 0s - loss: 2.4509 - accuracy: 0.5033
Epoch 1: val_accuracy improved from -inf to 0.56293, saving model to mobilenet_face.h5
355/355 [==============================] - 199s 556ms/step - loss: 2.4509 - accuracy: 0.5033 - val_loss: 1.1720 - val_accuracy: 0.5629
Epoch 2/3
355/355 [==============================] - ETA: 0s - loss: 2.0897 - accuracy: 0.5676
Epoch 2: val_accuracy improved from 0.56293 to 0.57328, saving model to mobilenet_face.h5

### Retraining the model for more epochs

In [16]:
start_epoch=len(hist1.history['loss'])
for l in base_model.layers:
    l.trainable=True
    
model.load_weights(net_description+'.h5')
model.compile(optimizer=Adam(lr=1e-4,decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

SECOND_EPOCHS=start_epoch+10
mc = ModelCheckpoint(net_description+'_ft.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
#es=EarlyStopping(monitor='val_accuracy',patience=2 )

hist2=model.fit(train_generator, steps_per_epoch=train_generator.samples//BATCH_SIZE, epochs=SECOND_EPOCHS, verbose=1, 
                    initial_epoch=start_epoch, validation_data=val_generator, validation_steps=val_generator.samples // BATCH_SIZE, callbacks=[mc],class_weight=class_weights)

Epoch 4/13
355/355 [==============================] - ETA: 0s - loss: 1.7821 - accuracy: 0.6239
Epoch 4: val_accuracy improved from -inf to 0.65014, saving model to mobilenet_face_ft.h5
355/355 [==============================] - 150s 416ms/step - loss: 1.7821 - accuracy: 0.6239 - val_loss: 0.9601 - val_accuracy: 0.6501
Epoch 5/13
355/355 [==============================] - ETA: 0s - loss: 1.5615 - accuracy: 0.6648
Epoch 5: val_accuracy improved from 0.65014 to 0.65761, saving model to mobilenet_face_ft.h5
355/355 [==============================] - 144s 405ms/step - loss: 1.5615 - accuracy: 0.6648 - val_loss: 0.9589 - val_accuracy: 0.6576
Epoch 6/13
355/355 [==============================] - ETA: 0s - loss: 1.4413 - accuracy: 0.6838
Epoch 6: val_accuracy improved from 0.65761 to 0.67213, saving model to mobilenet_face_ft.h5
355/355 [==============================] - 136s 382ms/step - loss: 1.4413 - accuracy: 0.6838 - val_loss: 0.9112 - val_accuracy: 0.6721
Epoch 7/13
355/355 [===========